In [0]:
import numpy as np
import cv2
from google.colab import files
import functools
import torch
import torch.nn as nn
import torch.nn.functional as F
import os.path as osp
import glob
import torch
import time
import copy

In [0]:
#Mounting Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
modelPath = 'gdrive/My Drive/ESRGAN/RRDB_ESRGAN_x4.pth'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
"""
Resizes images
parameters:
  image : conatins image
  size : list of the required dimention width,height
returns:
  resized image
"""
def resize(image,size):
    img = cv2.resize(image,(size[0],size[1]), interpolation = cv2.INTER_CUBIC)
    return(img)

In [0]:
"""
  Produces blur images
"""
def softBlurContours(image, contours, ksize, sigmaX, *args, **kwargs):
	iterations = 3
	if 'iters' in kwargs:
		iterations = kwargs['iters']
	sigmaY = args[0] if len(args) > 0 and args[0] != None else sigmaX
	mksize = args[1] if len(args) > 1 and args[1] != None else ksize
	msigmaX = args[2] if len(args) > 2 and args[2] != None else sigmaX
	msigmaY = args[3] if len(args) > 3 and args[3] != None else msigmaX
	mask = np.zeros(image.shape[:2])
	for i, contour in enumerate(contours):
		cv2.drawContours(mask, contour, 0, 255, -1)
	blurred_image = cv2.GaussianBlur(image, (ksize,ksize), sigmaX, None, sigmaY)
	result = np.copy(image)
	for _ in range(iterations):
		alpha = mask/255.
		result = alpha[:, :, None]*blurred_image + (1-alpha)[:, :, None]*result
		mask = cv2.GaussianBlur(mask, (mksize, mksize), msigmaX, None, msigmaY)
	return result

def softBlurRect(image, rect, ksize, sigmaX, *args, **kwargs):
	x,y,w,h = rect
	contours = [[np.array([[x,y],[x+w,y],[x+w,y+h],[x,y+h]])]]
	return softBlurContours(image, contours, ksize, sigmaX, *args, **kwargs)

def blurContours(image, contours, ksize, sigmaX, *args):
	sigmaY = args[0] if len(args) > 0 else sigmaX
	mask = np.zeros(image.shape[:2])
	for i, contour in enumerate(contours):
		cv2.drawContours(mask, contour, i, 255, -1)
	blurred_image = cv2.GaussianBlur(image, (ksize, ksize), sigmaX, None, sigmaY)
	result = np.copy(image)
	alpha = mask/255.
	result = alpha[:, :, None]*blurred_image + (1-alpha)[:, :, None]*result
	return result

def blurRect(image, rect, ksize, sigmaX, *args):
	x,y,w,h = rect
	contours = [[np.array([[x,y],[x+w,y],[x+w,y+h],[x,y+h]])]]
	return blurContours(image, contours, ksize, sigmaX, *args)

"""
Blurs the image outside the given portion
parameters:
  image: contains image
  portion: contains the coordinates of the region not to blur [x1,y1,x2,y2]
returns:
  image with blurred background
"""
def blurOut(image,portion):
    shape = image.shape
    x1,y1,x2,y2 = portion
    rect1 = [0,0,shape[1],y1]
    rect2 = [0,y2,shape[1],shape[0]-y2]
    rect3 = [0,y1,x1,y2-y1]
    rect4 = [x2,y1,shape[1]-x2,y2-y1]
    img1 = softBlurRect(image,rect1, 27, 9,9,55, iters = 7)
    img2 = softBlurRect(img1,rect2, 27, 9,9,55, iters = 7) 
    img3 = softBlurRect(img2,rect3, 27, 9,9,55, iters = 7) 
    img4 = softBlurRect(img3,rect4, 27, 9,9,55, iters = 7)
    return(img4)

In [0]:
"""
Superimposes the newImage over the image
parameters:
  image: background image to superimpose
  newImage: image to paste on the background image
returns:
  returns required image
"""
def superImpose(image,newImage):
    shape1 = image.shape
    shape2 = newImage.shape
    pip_h = int((shape1[0]-shape2[0])/2)
    pip_w = int((shape1[1]-shape2[1])/2)
    image[pip_h:pip_h+shape2[0],pip_w:pip_w+shape2[1]] = newImage
    return(image,pip_w,pip_h)

In [0]:
"""
uploads the required file
"""
def uploadFiles():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return (k)

In [0]:
"""
Defined model to upscale the image
"""
def make_layer(block, n_layers):
    layers = []
    for _ in range(n_layers):
        layers.append(block())
    return nn.Sequential(*layers)


class ResidualDenseBlock_5C(nn.Module):
    def __init__(self, nf=64, gc=32, bias=True):
        super(ResidualDenseBlock_5C, self).__init__()
        # gc: growth channel, i.e. intermediate channels
        self.conv1 = nn.Conv2d(nf, gc, 3, 1, 1, bias=bias)
        self.conv2 = nn.Conv2d(nf + gc, gc, 3, 1, 1, bias=bias)
        self.conv3 = nn.Conv2d(nf + 2 * gc, gc, 3, 1, 1, bias=bias)
        self.conv4 = nn.Conv2d(nf + 3 * gc, gc, 3, 1, 1, bias=bias)
        self.conv5 = nn.Conv2d(nf + 4 * gc, nf, 3, 1, 1, bias=bias)
        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

        # initialization
        # mutil.initialize_weights([self.conv1, self.conv2, self.conv3, self.conv4, self.conv5], 0.1)

    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        return x5 * 0.2 + x


class RRDB(nn.Module):
    '''Residual in Residual Dense Block'''

    def __init__(self, nf, gc=32):
        super(RRDB, self).__init__()
        self.RDB1 = ResidualDenseBlock_5C(nf, gc)
        self.RDB2 = ResidualDenseBlock_5C(nf, gc)
        self.RDB3 = ResidualDenseBlock_5C(nf, gc)

    def forward(self, x):
        out = self.RDB1(x)
        out = self.RDB2(out)
        out = self.RDB3(out)
        return out * 0.2 + x


class RRDBNet(nn.Module):
    def __init__(self, in_nc, out_nc, nf, nb, gc=32):
        super(RRDBNet, self).__init__()
        RRDB_block_f = functools.partial(RRDB, nf=nf, gc=gc)

        self.conv_first = nn.Conv2d(in_nc, nf, 3, 1, 1, bias=True)
        self.RRDB_trunk = make_layer(RRDB_block_f, nb)
        self.trunk_conv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        #### upsampling
        self.upconv1 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.upconv2 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.HRconv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.conv_last = nn.Conv2d(nf, out_nc, 3, 1, 1, bias=True)

        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        fea = self.conv_first(x)
        trunk = self.trunk_conv(self.RRDB_trunk(fea))
        fea = fea + trunk

        fea = self.lrelu(self.upconv1(F.interpolate(fea, scale_factor=2, mode='nearest')))
        fea = self.lrelu(self.upconv2(F.interpolate(fea, scale_factor=2, mode='nearest')))
        out = self.conv_last(self.lrelu(self.HRconv(fea)))

        return out

In [0]:
"""
upscales the given img X 4
parameters:
  img: contains the image to upscale
returns:
  upscaled image
"""
def upscale(img):
  device = torch.device('cpu')
  model = RRDBNet(3, 3, 64, 23, gc=32)
  model.load_state_dict(torch.load(modelPath), strict=True)
  model.eval()
  model = model.to(device)
  img = img * 1.0 / 255
  img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
  img_LR = img.unsqueeze(0)
  img_LR = img_LR.to(device)

  with torch.no_grad():
      output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
  output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
  output = (output * 255.0).round()
  return(output)

In [0]:
def download():
  files.download('result.jpg')

In [0]:
""" Main function """
flag = True
print(time.ctime())
height,width = 1000,1500
newHeight, newWidth = 0,0
imgName = uploadFiles()
im = cv2.imread(imgName)
shape = im.shape
aspectRatio = shape[1]/shape[0]
reqAspectRatio = width/height
if(shape[0]>=height and shape[1]>=width and (shape[0]!=height or shape[1]!=width)):
    reqHeight = height
    while(reqHeight > 0):
        newWidth = int(aspectRatio*reqHeight)
        newHeight = reqHeight
        if(newWidth <= width):
            break
        reqHeight = reqHeight -1
elif(shape[0]<=height and shape[1]<=width and (shape[0]!=height or shape[1]!=width)):
    reqHeight = height
    while(reqHeight > 0):
        newWidth = int(aspectRatio*reqHeight)
        newHeight = reqHeight
        if(newWidth <= width):
            break
        reqHeight = reqHeight - 1
elif(shape[0]>=height and shape[1]<=width and (shape[0]!=height or shape[1]!=width)):
    reqHeight = height
    while(reqHeight > 0):
        newWidth = int(aspectRatio*reqHeight)
        newHeight = height
        if(newWidth <= width):
            break
        reqHeight = reqHeight -1
elif(shape[0]<=height and shape[1]>=width and (shape[0]!=height or shape[1]!=width)):
    reqHeight = height
    while(reqHeight > 0):
        newWidth = int(aspectRatio*reqHeight)
        newHeight = reqHeight
        if(newWidth <= width):
            break    
        reqHeight = reqHeight - 1

else:
    newHeight = height
    newWidth = width
print('New Dimentions : ',newWidth,'X',newHeight)
if(newHeight <= shape[0] and newWidth <= shape[1]):
    #shrink
    img = resize(im,[newWidth,newHeight])
    background = resize(im,[width,height])
    image,stX,stY = superImpose(background,img)
    image = blurOut(image,[stX,stY,stX+img.shape[1],stY+img.shape[0]])
    diff = np.abs((width-newWidth)-(height-newHeight))
    if(diff > 600):
      print('Image Size Does not match')
      flag = False
    print('Difference : ',np.abs((width-newWidth)-(height-newHeight)))
    cv2.imwrite('result.jpg',image)    
else:
    reqRatio = newHeight/im.shape[0]
    n_x4 = int(np.floor(reqRatio/4))
    i = 0
    img = copy.deepcopy(im)
    while(i<n_x4):
        print('upscaling')
        img = upscale(img)
        i = i + 1
    #img = resize(img,[newWidth,newHeight])
    print('Difference : ',np.abs((width-newWidth)-(height-newHeight)))
    background = resize(im,[width,height])
    image,stX,stY = superImpose(background,img)
    image = blurOut(image,[stX,stY,stX+img.shape[1],stY+img.shape[0]])
    diff = np.abs((width-newWidth)-(height-newHeight))
    if(diff > 600):
      flag = False
      print('Image Size Does not match')
    cv2.imwrite('result.jpg',image)
print(time.ctime())
if(flag):
  download()

Mon Jul  1 17:29:11 2019


Saving IND0000786.jpg to IND0000786 (4).jpg
New Dimentions :  1500 X 320
Difference :  680
Image Size Does not match
Mon Jul  1 17:29:50 2019
